In [1]:
import os

In [2]:
pwd

'd:\\PW_DS\\YOLO\\End-To-End-Waste-Detection-Project-using-Yolo-v5\\research'

In [3]:
os.chdir('..')

In [4]:
pwd

'd:\\PW_DS\\YOLO\\End-To-End-Waste-Detection-Project-using-Yolo-v5'

## Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    yolov5_root: Path
    yolov5_models: Path
    custom_yolov5: Path
    train_path: Path
    yolov5_runs: Path
    history: Path
    git_URL: str
    IMAGE_SIZE: int
    BATCH_SIZE: int
    EPOCHS: int
    DATA: Path
    WEIGHTS: str
    NAME: str


## Config Manager

In [7]:
from waste_detection.utils.common import  read_yaml , create_directories
from waste_detection.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

In [8]:
class ConfigManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    
    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params.train

        create_directories([config.root_dir])
        create_directories([config.history])

        model_training_config = ModelTrainingConfig(
            root_dir = Path(config.root_dir),
            yolov5_root = Path(config.yolov5_root),
            yolov5_models = Path(config.yolov5_models),
            custom_yolov5 = Path(config.custom_yolov5),
            train_path = Path(config.train_path),
            yolov5_runs = Path(config.yolov5_runs),
            history = Path(config.history),
            git_URL = config.git_URL,
            IMAGE_SIZE = params.IMAGE_SIZE,
            BATCH_SIZE = params.BATCH_SIZE,
            EPOCHS = params.EPOCHS,
            DATA = Path(params.DATA),
            WEIGHTS = params.WEIGHTS,
            NAME = params.NAME
        )

        return model_training_config

## Components

In [9]:
import os,sys
from pathlib import Path
from waste_detection.utils.common import  read_yaml, read_modify_write_yaml

import torch

from waste_detection.logger import  logging
from waste_detection.exception import CustomException

import subprocess

import yaml

from  datetime import datetime

In [10]:
#   root_dir: artifacts/model_training
#   yolov5_models: artifacts/model_training/yolov5/models
#   custom_yolov5: artifacts/model_training/yolov5/models/custom_yolov5s.yaml
#   train_path: artifacts/model_training/yolov5/train.py
#   yolov5_runs: artifacts/model_training/yolov5/runs
#   history: artifacts/model_training
#   git_URL: https://github.com/ultralytics/yolov5
  

#     IMAGE_SIZE: 640
#   BATCH_SIZE: 5
#   EPOCHS: 2
#   DATA: artifacts\data_ingestion\features\data.yaml
#   WEIGHTS: yolov5s.pt
#   NAME: yolov5s_results


In [11]:
from pathlib import Path
Path('artifacts\data_ingestion\features\data.yaml')

WindowsPath('artifacts/data_ingestion\x0ceatures/data.yaml')

## change git repo of yolov5 to root dir. then check if it works or not

In [36]:
import os
class ModelTraining:
    def __init__(
        self,
        config: ModelTrainingConfig):
        self.config = config

    def train(self):
        try:
            # Specify the URL of the GitHub repository
            # repo_url = "https://github.com/username/repo.git"

            # Specify the directory where you want to clone the repository
            root = os.getcwd()
            clone_dir = os.path.join(root,self.config.yolov5_root)

            # Create the command to clone the repository
            # command = f"git clone {self.config.git_URL}"

            # Execute the command using os.system()
            # os.system(command)
                
        
        ## 2. Change data.yml file and save custom data.yml file in 
            yolov5_pt = self.config.WEIGHTS.split('.')[0] + '.yaml'
            yolov5_pt_path = os.path.join(self.config.yolov5_models, yolov5_pt)
            print(yolov5_pt, yolov5_pt_path)

            data = read_yaml(Path(self.config.DATA))
            data_pt = read_yaml(Path(yolov5_pt_path))

            data_pt.nc = data.nc

            with open(self.config.custom_yolov5, 'w') as f:
                yaml.dump(data_pt, f)

            print(data_pt, data_pt.nc)
            read_modify_write_yaml(yolov5_pt_path ,self.config.custom_yolov5, {'nc': data.nc})

            ## 3. Train the model
            # os.system(f'python {self.config.train_path} --img {self.config.IMAGE_SIZE} --batch {self.config.BATCH_SIZE} --epochs {self.config.EPOCHS} --data {os.path.abspath(self.config.DATA)} --weights {self.config.WEIGHTS} --name {self.config.NAME} --cache')
            
            command = f'python {self.config.train_path} --img {self.config.IMAGE_SIZE} --batch {self.config.BATCH_SIZE} --epochs {self.config.EPOCHS} --data {os.path.abspath(self.config.DATA)} --weights {self.config.WEIGHTS} --name {self.config.NAME} --cache'
            result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print(result)
            
            crt_time = 'runs_' + datetime.now().strftime('%d_%m_%Y_%H_%M_%S') + '/'
            runs_path = os.path.join(self.config.history, crt_time)

            yolov5_runs = os.path.join(self.config.yolov5_models,'runs')

            # print(yolov5_runs)      
            
            create_directories([runs_path])

            # best_pt = os.path.join(yolov5_runs, 'train/yolov5s_results/weights/best.pt')          

            os.system(f'cp {yolov5_runs} {runs_path}')


            
    
        except Exception as e:
            raise CustomException(e,sys)



In [13]:
# class ModelTraining:
#     def __init__(
#         self,
#         config: ModelTrainingConfig):
#         self.config = config
    

#     def train(self):

#         try:
#             # 1. Clone git repo to root dir.
#             # command = f'git clone {self.config.git_URL}'
#             # result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
        
#             # 2. read yaml file then create a new yaml file of custom model
            # yolov5_pt = self.config.WEIGHTS.split('.')[0] + '.yaml'
            # yolov5_pt_path = os.path.join(self.config.yolov5_models, yolov5_pt)
            # print(yolov5_pt, yolov5_pt_path)

            # data = read_yaml(Path(self.config.DATA))
            # data_pt = read_yaml(Path(yolov5_pt_path))

            # data_pt.nc = data.nc

            # with open(self.config.custom_yolov5, 'w') as f:
            #     yaml.dump(data_pt, f)

#             # print(data_pt, data_pt.nc)
#             # read_modify_write_yaml(yolov5_pt_path ,self.config.custom_yolov5, {'nc': data.nc})

#             # 3. train, save the runs in history folder

#                 # crt_time = 'runs_' + datetime.now().strftime('%d_%m_%Y_%H_%M_%S') + '/'
#                 # runs_path = os.path.join(self.config.history, crt_time)
#                 # print(runs_path)

#                 # create_directories([runs_path])

#                 # best_pt = os.path.join(self.config.yolov5_runs, 'train/yolov5s_results/weights/best.pt')

# # ---------------------------------------------------------------------------------------------------------
#                 # weights_directory = str(self.config.yolov5_root)

#                 # # Ensure the directory exists
#                 # os.makedirs(weights_directory, exist_ok=True)

#                 # # URL of the pre-trained weights file
#                 # weights_url = "https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt"

#                 # # Command to download the weights using wget
#                 # download_command = f'wget -O {os.path.join(weights_directory, "yolov5s.pt")} {weights_url}'

#                 # # Run the download command using subprocess
#                 # result = subprocess.run(download_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# # --------------------------------------------------------------------------------------------------------------------------------


#                 crt_dir = os.getcwd()

#                 command = f'python {self.config.train_path} --img {self.config.IMAGE_SIZE} --batch {self.config.BATCH_SIZE} --epochs {self.config.EPOCHS} --data {os.path.abspath(self.config.DATA)} --weights {self.config.WEIGHTS} --name {self.config.NAME} --cache'
#                 result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
#                 print(result)

#             # -----------------------------------------------------------
#                 command1 = f'cp {self.config.yolov5_runs} {runs_path}'
#                 result1 = subprocess.run(command1, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
#                 print(result1)

#                 command2 = f'cp {best_pt} {self.config.root_dir}'
#                 result2 = subprocess.run(command2, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
#                 print(result2)

#                 command3 = f'rm -rf {self.config.yolov5_runs}'
#                 result3 = subprocess.run(command3, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
#                 print(result3)


#             # os.system("cp {self.config.yolov5_runs} {runs_path}")
#             # os.system("cp {best_pt} {self.config.root_dir}")
            
#             # os.system("rm -rf {self.config.yolov5_runs}")

        
#         except Exception as e:
#             raise CustomException(e,sys)

## Pipeline

In [37]:
try:
    config = ConfigManager()
    model_training = ModelTraining(config.get_model_training_config())
    model_training.train()

except Exception as e:
    raise CustomException(e,sys)

yolov5s.yaml yolov5\models\yolov5s.yaml
{'nc': 13, 'depth_multiple': 0.33, 'width_multiple': 0.5, 'anchors': [[10, 13, 16, 30, 33, 23], [30, 61, 62, 45, 59, 119], [116, 90, 156, 198, 373, 326]], 'backbone': [[-1, 1, 'Conv', [64, 6, 2, 2]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 3, 'C3', [128]], [-1, 1, 'Conv', [256, 3, 2]], [-1, 6, 'C3', [256]], [-1, 1, 'Conv', [512, 3, 2]], [-1, 9, 'C3', [512]], [-1, 1, 'Conv', [1024, 3, 2]], [-1, 3, 'C3', [1024]], [-1, 1, 'SPPF', [1024, 5]]], 'head': [[-1, 1, 'Conv', [512, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 6], 1, 'Concat', [1]], [-1, 3, 'C3', [512, False]], [-1, 1, 'Conv', [256, 1, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 4], 1, 'Concat', [1]], [-1, 3, 'C3', [256, False]], [-1, 1, 'Conv', [256, 3, 2]], [[-1, 14], 1, 'Concat', [1]], [-1, 3, 'C3', [512, False]], [-1, 1, 'Conv', [512, 3, 2]], [[-1, 10], 1, 'Concat', [1]], [-1, 3, 'C3', [1024, False]], [[17, 20, 23], 1, 'Detect', ['nc', 'anchors']]]} 13
Completed

In [ ]:
crt_dir = os.getcwd()

chdir = 


command = f'cd {self.config.yolov5_root} && python train.py --img {self.config.IMAGE_SIZE} --batch {self.config.BATCH_SIZE} --epochs {self.config.EPOCHS} --data {os.path.join(crt_dir, self.config.DATA)} --weights {self.config.WEIGHTS} --name {self.config.NAME} --cache'

SyntaxError: invalid syntax (845190903.py, line 3)